In [1]:
import os, glob
import numpy as np
import keras
import tensorflow as tf
from keras.models import load_model

Using TensorFlow backend.


In [2]:
os.chdir('/home/user3/anaconda3/')

model_stft_reg = load_model('STFT_2DCNN_1DCNN_regression_multi.h5')
model_mrcg_reg = load_model('MRCG_2DCNN_1DCNN_regression_multi.h5')
model_stft_clf = load_model('STFT_2DCNN_1DCNN_clf_multi.h5')
model_mrcg_clf = load_model('MRCG_2DCNN_1DCNN_clf_multi.h5')

In [3]:
import librosa
os.chdir('/home/user3/anaconda3/VAD_DNN/')
import MRCG as mrcg

In [4]:
def mag_phase(file_path, sr=48000, n_fft=1024, hop_length=512, db=False, n_mels=50) :
    audio, sr = librosa.load(file_path, sr=sr, mono=False)   # 원래의 sr, stereo
    DL = librosa.stft(audio[0], n_fft=n_fft, hop_length=hop_length)
    DL_mag, DL_phase = librosa.magphase(DL)
    
    DR = librosa.stft(audio[1], n_fft=n_fft, hop_length=hop_length)
    DR_mag, DR_phase = librosa.magphase(DR)
    
    if db :
        DL_mag = librosa.core.amplitude_to_db(DL_mag)
        DR_mag = librosa.core.amplitude_to_db(DR_mag)
     
    # rescale the right magnitudes w.r.t left channel magnitude 
    avg = DL_mag.mean()     
    stdv = DL_mag.std()
    DL_mag = (DL_mag - avg)/stdv
    DR_mag = (DR_mag - avg)/stdv
    
    return( (DL_mag[1:, :], np.angle(DL_phase)[1:, :]), (DR_mag[1:, :], np.angle(DR_phase)[1:, :]) )

In [5]:
def mrcg_transpose(file_path, sr=48000):
    X, sample_rate = librosa.load(file_path, sr=sr, mono=False)
    mrcg_L = mrcg.mrcg_extract(X[0], sample_rate)
    mrcg_R = mrcg.mrcg_extract(X[1], sample_rate)
    
    return mrcg_L, mrcg_R

In [6]:
def generate_instances(array_2d, seq_len, hop):
    """
    array_2d : ndarray.  STFT magnitude or phase.
    dest_path : file path
    seq_len : number of frames in an instance.
    label : segmented labels.  0 and 1's. The same length as original wav file of the audio sample. 
    """
    row_size, col_size = array_2d.shape[0], array_2d.shape[1]
    stack_array = []    # 4D tensor that will hold the instances

    j=0
    while j <= (col_size - (seq_len+1)): 
        context_frame = array_2d[:, j:(j+seq_len)]
        
        stack_array.append(context_frame[:,:,np.newaxis])   # make context_frame to 3d tensor & append          
        j = j+hop
        
    return np.stack(stack_array, axis=0)

In [17]:
def stft_predict(model1, model2, file_path):
    (mag_L, phase_L), (mag_R, phase_R) = mag_phase(file_path, sr=48000, db=True)
    mag_L_stack = generate_instances(mag_L, 100, 10) 
    mag_R_stack = generate_instances(mag_R, 100, 10)
    phase_L_stack = generate_instances(phase_L, 100, 10)
    phase_R_stack = generate_instances(phase_R, 100, 10)
    X = np.concatenate([mag_L_stack, phase_L_stack, mag_R_stack, phase_R_stack], axis = -1)
    vad_pred_1, pred_1 = model1.predict(X)
    vad_pred_1 = np.where(vad_pred_1 >= 0.5, 1, 0)

    vad_pred_2, pred_2 = model2.predict(X)
    vad_pred_2 = np.where(vad_pred_2 >= 0.5, 1, 0)
    pred_2 = np.argmax(pred_2, axis = 1)
    return vad_pred_1, pred_1, vad_pred_2, pred_2

In [18]:
def mrcg_predict(model1, model2, file_path):
    mrcg_L, mrcg_R = mrcg_transpose(file_path, sr=48000)
    mrcg_L_stack = generate_instances(mrcg_L, 100, 10)
    mrcg_R_stack = generate_instances(mrcg_R, 100, 10)
    X = np.concatenate([mrcg_L_stack, mrcg_R_stack], axis=-1)
    vad_pred_1, pred_1 = model1.predict(X)
    vad_pred_1 = np.where(vad_pred_1 >= 0.5, 1, 0)

    vad_pred_2, pred_2 = model2.predict(X)
    vad_pred_2 = np.where(vad_pred_2 >= 0.5, 1, 0)
    pred_2 = np.argmax(pred_2, axis = 1)
    return vad_pred_1, pred_1, vad_pred_2, pred_2

In [12]:
def class_instances_count(sample_instances_labels) : 
    """
    sample_instances_labels : class labels for the instances
    returns a list where elements are : (class_label, count)
    """
    # sample_instances_labels : class labels for the instances 
    import operator
    unique, counts = np.unique(sample_instances_labels, return_counts=True)
    dict_temp = dict(zip(unique, counts))
    return sorted(dict_temp.items(), key=operator.itemgetter(1), reverse=True)  

In [46]:
def predict(model1, model2, file_path, threshold = 0.1):
    vad_pred, pred = stft_predict(model1, file_path)
    if(vad_pred.mean() < threshold):
        return 0
    else:
        vad_pred, pred = stft_predict(model2, file_path)
        
        for i in range(0, len(pred)):
            if(vad_pred[i] == 0):
                pred[i] = 0
            elif(pred[i] < -80):
                pred[i] = -90
            elif((pred[i] >= -80) & (pred[i] < -60)):
                pred[i] = -70
            elif((pred[i] >= -60) & (pred[i] < -40)):
                pred[i] = -50
            elif((pred[i] >= -40) & (pred[i] < -20)):
                pred[i] = -30
            elif((pred[i] >= -20) & (pred[i] < 0)):
                pred[i] = -10
            elif((pred[i] >= 0) & (pred[i] < 20)):
                pred[i] = 10
            elif((pred[i] >= 20) & (pred[i] < 40)):
                pred[i] = 30
            elif((pred[i] >= 40) & (pred[i] < 60)):
                pred[i] = 50
            elif((pred[i] >= 60) & (pred[i] < 80)):
                pred[i] = 70
            elif(pred[i] >= 80):
                pred[i] = 90
        '''
        for i in range(0, len(pred)):
            if(vad_pred[i] == 0):
                pred[i] = 0
            elif(pred[i] == 0):
                pred[i] = 0
            elif(pred[i] == 1):
                pred[i] = -90
            elif(pred[i] == 2):
                pred[i] = -70
            elif(pred[i] == 3):
                pred[i] = -50
            elif(pred[i] == 4):
                pred[i] = -30
            elif(pred[i] == 5):
                pred[i] = -10
            elif(pred[i] == 6):
                pred[i] = 10
            elif(pred[i] == 7):
                pred[i] = 30
            elif(pred[i] == 8):
                pred[i] = 50
            elif(pred[i] == 9):
                pred[i] = 70
            elif(pred[i] == 10):
                pred[i] = 90
        '''
        
        pred = pred[np.array(vad_pred.reshape(pred.shape)) == 1]
        
        if(pred.sum() == 0):
            return 0
    
        for class_label, class_count in class_instances_count(pred) :
            if(class_label != 0):
                return class_label

In [23]:
def predict(model1, model2, model3, model4, file_path, threshold = 0.1):
    vad_pred_1, pred_1, vad_pred_2, pred_2 = stft_predict(model1, model2, file_path)
    vad_pred_3, pred_3, vad_pred_4, pred_4 = mrcg_predict(model3, model4, file_path)
    vad_pred_mean = (vad_pred_1.mean() + vad_pred_2.mean() + vad_pred_3.mean() + vad_pred_4.mean()) / 4
     
    if(vad_pred_mean < threshold):
        return 0
    else:
        for i in range(0, len(pred_1)):
            if(vad_pred_1[i] == 0):
                pred_1[i] = 0
            elif(pred_1[i] < -80):
                pred_1[i] = -90
            elif((pred_1[i] >= -80) & (pred_1[i] < -60)):
                pred_1[i] = -70
            elif((pred_1[i] >= -60) & (pred_1[i] < -40)):
                pred_1[i] = -50
            elif((pred_1[i] >= -40) & (pred_1[i] < -20)):
                pred_1[i] = -30
            elif((pred_1[i] >= -20) & (pred_1[i] < 0)):
                pred_1[i] = -10
            elif((pred_1[i] >= 0) & (pred_1[i] < 20)):
                pred_1[i] = 10
            elif((pred_1[i] >= 20) & (pred_1[i] < 40)):
                pred_1[i] = 30
            elif((pred_1[i] >= 40) & (pred_1[i] < 60)):
                pred_1[i] = 50
            elif((pred_1[i] >= 60) & (pred_1[i] < 80)):
                pred_1[i] = 70
            elif(pred_1[i] >= 80):
                pred_1[i] = 90
                
        for i in range(0, len(pred_3)):
            if(vad_pred_3[i] == 0):
                pred_3[i] = 0
            elif(pred_3[i] < -80):
                pred_3[i] = -90
            elif((pred_3[i] >= -80) & (pred_3[i] < -60)):
                pred_3[i] = -70
            elif((pred_3[i] >= -60) & (pred_3[i] < -40)):
                pred_3[i] = -50
            elif((pred_3[i] >= -40) & (pred_3[i] < -20)):
                pred_3[i] = -30
            elif((pred_3[i] >= -20) & (pred_3[i] < 0)):
                pred_3[i] = -10
            elif((pred_3[i] >= 0) & (pred_3[i] < 20)):
                pred_3[i] = 10
            elif((pred_3[i] >= 20) & (pred_3[i] < 40)):
                pred_3[i] = 30
            elif((pred_3[i] >= 40) & (pred_3[i] < 60)):
                pred_3[i] = 50
            elif((pred_3[i] >= 60) & (pred_3[i] < 80)):
                pred_3[i] = 70
            elif(pred_3[i] >= 80):
                pred_3[i] = 90
        
        for i in range(0, len(pred_2)):
            if(vad_pred_2[i] == 0):
                pred_2[i] = 0
            elif(pred_2[i] == 0):
                pred_2[i] = 0
            elif(pred_2[i] == 1):
                pred_2[i] = -90
            elif(pred_2[i] == 2):
                pred_2[i] = -70
            elif(pred_2[i] == 3):
                pred_2[i] = -50
            elif(pred_2[i] == 4):
                pred_2[i] = -30
            elif(pred_2[i] == 5):
                pred_2[i] = -10
            elif(pred_2[i] == 6):
                pred_2[i] = 10
            elif(pred_2[i] == 7):
                pred_2[i] = 30
            elif(pred_2[i] == 8):
                pred_2[i] = 50
            elif(pred_2[i] == 9):
                pred_2[i] = 70
            elif(pred_2[i] == 10):
                pred_2[i] = 90
        
        for i in range(0, len(pred_4)):
            if(vad_pred_4[i] == 0):
                pred_4[i] = 0
            elif(pred_4[i] == 0):
                pred_4[i] = 0
            elif(pred_4[i] == 1):
                pred_4[i] = -90
            elif(pred_4[i] == 2):
                pred_4[i] = -70
            elif(pred_4[i] == 3):
                pred_4[i] = -50
            elif(pred_4[i] == 4):
                pred_4[i] = -30
            elif(pred_4[i] == 5):
                pred_4[i] = -10
            elif(pred_4[i] == 6):
                pred_4[i] = 10
            elif(pred_4[i] == 7):
                pred_4[i] = 30
            elif(pred_4[i] == 8):
                pred_4[i] = 50
            elif(pred_4[i] == 9):
                pred_4[i] = 70
            elif(pred_4[i] == 10):
                pred_4[i] = 90
        
        pred_1 = pred_1[np.array(vad_pred_1.reshape(pred_1.shape)) == 1]
        pred_2 = pred_2[np.array(vad_pred_2.reshape(pred_2.shape)) == 1]
        pred_3 = pred_3[np.array(vad_pred_3.reshape(pred_3.shape)) == 1]
        pred_4 = pred_4[np.array(vad_pred_4.reshape(pred_4.shape)) == 1]
        pred_1 = pred_1.reshape(1, -1)
        pred_2 = pred_2.reshape(1, -1)
        pred_3 = pred_3.reshape(1, -1)
        pred_4 = pred_4.reshape(1, -1)
        
        pred = np.concatenate([pred_1, pred_2, pred_3, pred_4], axis = -1)
        
        if(pred.sum() == 0):
            return 0
    
        for class_label, class_count in class_instances_count(pred) :
            if(class_label != 0):
                return class_label

In [24]:
sse = 0
cnt = 0

In [25]:
sample_data_repo_0 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '0도')
samples_0 = glob.glob(os.path.join(sample_data_repo_0, '**', '*wav'), recursive=True)
samples_0 = sorted(samples_0)   # sort the samples
n_0 = len(samples_0)

for i in range(0, n_0):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_0[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (-90 - pred) ** 2

fftfilter
6.247765999999999
gamma total
8.583325000000002
coch1
3.221884000000003
coch2
3.272469000000001
get avg
0.02738700000000449
fftfilter
6.043246999999994
gamma total
8.15383
coch1
3.206347000000008
coch2
3.2692320000000024
get avg
0.027152000000000953
fftfilter
3.7717810000000043
gamma total
4.686247000000009
coch1
1.2906059999999826
coch2
1.8754440000000159
get avg
0.24019800000002078
fftfilter
3.1539140000000145
gamma total
4.498583999999994
coch1
1.5935039999999958
coch2
2.1994899999999973
get avg
0.2420189999999991
fftfilter
6.092536999999993
gamma total
7.9661020000000065
coch1
2.7409210000000144
coch2
3.228905999999995
get avg
0.024699999999995725
fftfilter
6.503161000000006
gamma total
8.385802000000012
coch1
2.8450750000000085
coch2
3.2235089999999786
get avg
0.025022000000006983
fftfilter
7.071258
gamma total
9.577782999999982
coch1
3.275092000000001
coch2
3.360515000000021
get avg
0.06536200000002168
fftfilter
7.115122000000014
gamma total
9.674738999999988
coch1
3.26

In [26]:
sample_data_repo_20 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '20도')
samples_20 = glob.glob(os.path.join(sample_data_repo_20, '**', '*wav'), recursive=True)
samples_20 = sorted(samples_20)   # sort the samples
n_20 = len(samples_20)

for i in range(0, n_20):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_20[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (-70 - pred) ** 2

fftfilter
8.497148000000038
gamma total
11.445300000000032
coch1
3.443979000000013
coch2
3.4073119999999903
get avg
0.10706200000004173
fftfilter
8.665537000000029
gamma total
11.627203000000009
coch1
3.3031050000000164
coch2
3.4330909999999903
get avg
0.07434000000000651
fftfilter
9.55433400000004
gamma total
12.305143000000044
coch1
3.3639459999999417
coch2
3.4177369999999883
get avg
0.0767999999999347
fftfilter
8.875113000000056
gamma total
11.624351000000047
coch1
3.280803999999989
coch2
3.3763609999999744
get avg
0.0917459999999437
fftfilter
9.957843000000025
gamma total
12.72412200000008
coch1
3.3016609999999673
coch2
3.3946320000000014
get avg
0.07521299999996245
fftfilter
8.85103700000002
gamma total
11.60775799999999
coch1
3.273368000000005
coch2
3.4545249999999896
get avg
0.07240000000001601
fftfilter
8.372420000000034
gamma total
11.066650999999979
coch1
3.336595999999986
coch2
3.586651999999958
get avg
0.07497999999998228
fftfilter
9.159743000000049
gamma total
11.866589999

fftfilter
8.820874000000003
gamma total
11.613063999999895
coch1
3.3346220000000812
coch2
3.4657999999999447
get avg
0.07581900000013775
fftfilter
8.958282999999938
gamma total
11.898275000000012
coch1
3.304380999999921
coch2
3.435513000000128
get avg
0.07387500000004366
fftfilter
9.464165999999977
gamma total
12.2589210000001
coch1
3.3573999999998705
coch2
3.432612000000063
get avg
0.10565799999994852
fftfilter
9.167523999999958
gamma total
11.983868999999913
coch1
3.325311000000056
coch2
3.446713000000045
get avg
0.10327600000005077
fftfilter
9.139274999999998
gamma total
11.96360999999979
coch1
3.3394110000001547
coch2
3.463706000000002
get avg
0.10780700000009347
fftfilter
9.260114000000158
gamma total
12.01847399999997
coch1
3.3424580000000788
coch2
3.4799209999998766
get avg
0.10354600000005121
fftfilter
8.668267999999898
gamma total
11.453348000000005
coch1
3.326224999999795
coch2
3.4665200000001732
get avg
0.07711800000015501
fftfilter
9.263425000000097
gamma total
12.000105000

In [27]:
sample_data_repo_40 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '40도')
samples_40 = glob.glob(os.path.join(sample_data_repo_40, '**', '*wav'), recursive=True)
samples_40 = sorted(samples_40)   # sort the samples
n_40 = len(samples_40)

for i in range(0, n_40):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_40[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (-50 - pred) ** 2

fftfilter
8.791654999999992
gamma total
11.529418999999962
coch1
3.333523000000014
coch2
3.4717400000001817
get avg
0.08106199999997443
fftfilter
8.766851999999972
gamma total
11.50340600000004
coch1
3.324098999999933
coch2
3.4596059999998943
get avg
0.07545600000003105
fftfilter
8.051558999999997
gamma total
10.784623999999894
coch1
3.319884000000002
coch2
3.4556800000000294
get avg
0.07517600000005586
fftfilter
7.894078000000036
gamma total
10.613292000000001
coch1
3.3165950000000066
coch2
3.4242259999998623
get avg
0.07561800000007679
fftfilter
8.17275799999993
gamma total
10.906439999999975
coch1
3.323733000000175
coch2
3.4321860000000015
get avg
0.07398699999998826
fftfilter
9.089005000000043
gamma total
11.856262999999899
coch1
3.2965770000000703
coch2
3.4355900000000474
get avg
0.07592999999997119
fftfilter
9.43448699999999
gamma total
12.512370000000146
coch1
3.3426899999999478
coch2
3.3892359999999826
get avg
0.07415799999989758
fftfilter
9.272518999999875
gamma total
12.30903

fftfilter
9.117130999999972
gamma total
11.986996999999974
coch1
3.333748000000014
coch2
3.458476000000246
get avg
0.07359699999960867
fftfilter
8.161215999999968
gamma total
10.885007000000314
coch1
3.3274099999998725
coch2
3.4526919999998427
get avg
0.07625699999971403
fftfilter
8.102178999999978
gamma total
10.846454999999878
coch1
3.314241000000038
coch2
3.430609999999888
get avg
0.0767030000001796
fftfilter
7.937577000000147
gamma total
10.849904000000151
coch1
3.3136279999998806
coch2
3.434049999999843
get avg
0.0747350000001461
fftfilter
8.67768100000012
gamma total
11.616691000000174
coch1
3.337728999999854
coch2
3.4679869999999937
get avg
0.07876400000031936
fftfilter
9.003813000000264
gamma total
11.67788900000005
coch1
3.3458499999997002
coch2
3.4639569999999367
get avg
0.07520199999999022
fftfilter
9.214567999999872
gamma total
11.884174000000257
coch1
3.332429999999931
coch2
3.467059000000063
get avg
0.07353199999988647
fftfilter
8.932980000000043
gamma total
11.6768250000

In [28]:
sample_data_repo_60 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '60도')
samples_60 = glob.glob(os.path.join(sample_data_repo_60, '**', '*wav'), recursive=True)
samples_60 = sorted(samples_60)   # sort the samples
n_60 = len(samples_60)

for i in range(0, n_60):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_60[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (-30 - pred) ** 2

fftfilter
6.793974000000162
gamma total
8.93894999999975
coch1
2.596385000000282
coch2
3.2396699999999328
get avg
0.028201000000080967
fftfilter
6.229964999999993
gamma total
8.391723000000184
coch1
3.2245680000000903
coch2
3.276340999999775
get avg
0.027505999999902997
fftfilter
7.94381599999997
gamma total
10.551143000000138
coch1
3.051481999999851
coch2
3.295734000000266
get avg
0.08280700000022989
fftfilter
8.110642000000098
gamma total
10.653859999999895
coch1
3.2430960000001505
coch2
3.317081999999573
get avg
0.06259300000010626
fftfilter
6.938090000000102
gamma total
9.231994000000213
coch1
3.2408439999999246
coch2
3.2987080000002607
get avg
0.04481400000031499
fftfilter
7.728317000000061
gamma total
10.215307999999823
coch1
3.2393050000000585
coch2
3.320686999999907
get avg
0.03026899999986199
fftfilter
9.546722000000045
gamma total
12.406449000000066
coch1
3.3207820000002357
coch2
3.4667829999998503
get avg
0.10743399999955727
fftfilter
9.990979000000152
gamma total
12.7406329

In [29]:
sample_data_repo_80 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '80도')
samples_80 = glob.glob(os.path.join(sample_data_repo_80, '**', '*wav'), recursive=True)
samples_80 = sorted(samples_80)   # sort the samples
n_80 = len(samples_80)

for i in range(0, n_80):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_80[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (-10 - pred) ** 2

fftfilter
9.347736000000168
gamma total
12.187479000000167
coch1
3.3335259999998925
coch2
3.4469990000002326
get avg
0.07416300000022602
fftfilter
8.216968000000179
gamma total
10.967311000000336
coch1
3.315490999999838
coch2
3.453787999999804
get avg
0.07677400000011403
fftfilter
9.023593999999775
gamma total
11.78370300000006
coch1
3.3285949999999502
coch2
3.445521000000099
get avg
0.0749839999998585
fftfilter
9.137767000000167
gamma total
11.944351000000097
coch1
3.323859000000084
coch2
3.4471020000000863
get avg
0.1022769999999582
fftfilter
9.14802200000031
gamma total
12.07959899999969
coch1
3.320006000000376
coch2
3.4228049999996983
get avg
0.07427200000029188
fftfilter
8.352122000000236
gamma total
11.113542000000052
coch1
3.3165469999999004
coch2
3.4555570000002263
get avg
0.07490699999971184
fftfilter
8.037036000000171
gamma total
10.752830999999787
coch1
3.3093950000002224
coch2
3.443647999999939
get avg
0.07406500000024607
fftfilter
9.79289800000015
gamma total
12.6013780000

fftfilter
8.459130999999616
gamma total
11.20810100000017
coch1
3.3233070000005682
coch2
3.461475000000064
get avg
0.07513999999991938
fftfilter
9.340049000000363
gamma total
12.09034900000006
coch1
3.3333849999999074
coch2
3.456770000000688
get avg
0.1104670000004262
fftfilter
9.092865999999958
gamma total
11.910179999999855
coch1
3.3310330000003887
coch2
3.469808999999259
get avg
0.07753499999944324
fftfilter
7.889940999999453
gamma total
10.607928999999785
coch1
3.318478000000141
coch2
3.4540229999993244
get avg
0.07611799999995128
fftfilter
8.357402999999977
gamma total
11.121728000000076
coch1
3.3241769999995086
coch2
3.456719000000703
get avg
0.07395100000030652
fftfilter
9.178130999999667
gamma total
11.95746599999984
coch1
3.323224000000664
coch2
3.4714340000000448
get avg
0.08234499999980471
fftfilter
10.91761200000019
gamma total
13.90475300000071
coch1
3.313130999999885
coch2
3.4489429999994172
get avg
0.07638000000042666
fftfilter
8.684401999999864
gamma total
11.4239139999

In [30]:
sample_data_repo_100 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '100도')
samples_100 = glob.glob(os.path.join(sample_data_repo_100, '**', '*wav'), recursive=True)
samples_100 = sorted(samples_100)   # sort the samples
n_100 = len(samples_100)

for i in range(0, n_100):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_100[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (10 - pred) ** 2

fftfilter
9.285961000000498
gamma total
12.030052000000069
coch1
3.35326299999997
coch2
3.4088629999996556
get avg
0.07725200000004406
fftfilter
9.219354999999268
gamma total
12.174793000000136
coch1
3.3130270000001474
coch2
3.4230749999996988
get avg
0.07574400000066817
fftfilter
8.055548000000272
gamma total
10.764766000000236
coch1
3.331344999999601
coch2
3.440312000000631
get avg
0.07574699999986478
fftfilter
9.316531000000396
gamma total
12.078177999999753
coch1
3.3413179999997737
coch2
3.458752999999888
get avg
0.07443100000000413
fftfilter
7.940860000000612
gamma total
10.659451999999874
coch1
3.3197339999996984
coch2
3.4600060000002486
get avg
0.07325899999977992
fftfilter
8.54382699999951
gamma total
11.330646999999772
coch1
3.3404239999999845
coch2
3.4517580000001544
get avg
0.07376600000043254
fftfilter
9.096674000000348
gamma total
11.832822999999735
coch1
3.357930000000124
coch2
3.4432200000001103
get avg
0.0748169999997117
fftfilter
9.204066000000239
gamma total
12.144296

fftfilter
8.980289000000084
gamma total
11.747478999999657
coch1
3.341395999999804
coch2
3.453486000000339
get avg
0.07413899999937712
fftfilter
8.974262000000635
gamma total
11.70182400000067
coch1
3.348544999999831
coch2
3.4131649999999354
get avg
0.10566799999924115
fftfilter
9.246313000000555
gamma total
12.030732000000171
coch1
3.3455599999997503
coch2
3.4710379999996803
get avg
0.07626199999958772
fftfilter
9.754898999999568
gamma total
12.546065999999882
coch1
3.3293790000006993
coch2
3.461338999999498
get avg
0.10414000000037049
fftfilter
8.309266000000207
gamma total
11.254961999999978
coch1
3.3160049999996772
coch2
3.4506689999998343
get avg
0.10830100000021048
fftfilter
8.804508000000169
gamma total
11.555043999999725
coch1
3.3580590000001393
coch2
3.408198000000084
get avg
0.07776999999987311
fftfilter
9.403666999999587
gamma total
12.416822000000138
coch1
3.2660909999995056
coch2
3.3670640000000276
get avg
0.07438899999942805
fftfilter
7.862793000000238
gamma total
10.5869

In [31]:
sample_data_repo_120 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '120도')
samples_120 = glob.glob(os.path.join(sample_data_repo_120, '**', '*wav'), recursive=True)
samples_120 = sorted(samples_120)   # sort the samples
n_120 = len(samples_120)

for i in range(0, n_120):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_120[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (30 - pred) ** 2

fftfilter
4.527258999999503
gamma total
5.499421000000439
coch1
2.3743689999992057
coch2
3.0379370000000563
get avg
0.15009600000030332
fftfilter
4.7407109999994645
gamma total
5.828714000000218
coch1
2.4538979999997537
coch2
3.0215950000001612
get avg
0.16559799999959068
fftfilter
7.2611720000004425
gamma total
9.578574000000117
coch1
3.2521120000001247
coch2
3.343001999999615
get avg
0.030846000000565255
fftfilter
7.295554000000266
gamma total
9.863268999999491
coch1
3.255852000000232
coch2
3.316413999999895
get avg
0.029750000000603904
fftfilter
5.47631199999978
gamma total
7.342751000000135
coch1
2.308839000000262
coch2
3.101434000000154
get avg
0.10291799999959039
fftfilter
5.97912499999984
gamma total
7.928527000000031
coch1
2.9052380000002813
coch2
3.2251829999995607
get avg
0.022910000000592845
fftfilter
6.811921999999868
gamma total
9.07133199999953
coch1
3.2241149999999834
coch2
3.282361000000492
get avg
0.02893000000040047
fftfilter
8.165439999999762
gamma total
10.463654999

In [32]:
sample_data_repo_140 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '140도')
samples_140 = glob.glob(os.path.join(sample_data_repo_140, '**', '*wav'), recursive=True)
samples_140 = sorted(samples_140)   # sort the samples
n_140 = len(samples_140)

for i in range(0, n_140):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_140[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (50 - pred) ** 2

fftfilter
8.828607000000375
gamma total
11.637620000000425
coch1
3.2731989999992948
coch2
3.432727000000341
get avg
0.1035859999992681
fftfilter
10.00095599999986
gamma total
13.081474999999955
coch1
3.378649999999652
coch2
3.6429760000000897
get avg
0.09053300000050513
fftfilter
8.973367999999937
gamma total
11.73990699999922
coch1
3.357785999999578
coch2
3.4286920000013197
get avg
0.10923000000002503
fftfilter
9.286767999999938
gamma total
12.1008810000003
coch1
3.3336230000004434
coch2
3.4516609999991488
get avg
0.07255800000166346
fftfilter
9.018039000000499
gamma total
11.759366999998747
coch1
3.297338000000309
coch2
3.4330530000006547
get avg
0.07428799999979674
fftfilter
9.447808000000805
gamma total
12.24710800000139
coch1
3.2793449999990116
coch2
3.4390999999995984
get avg
0.08899300000121002
fftfilter
8.997655999999552
gamma total
11.725495999999112
coch1
3.3321190000006027
coch2
3.469184999999925
get avg
0.10339899999962654
fftfilter
8.87746800000059
gamma total
11.866844999

fftfilter
8.161274000000049
gamma total
10.975612999998702
coch1
3.3285759999998845
coch2
3.453326000000743
get avg
0.07507000000077824
fftfilter
8.001431999999113
gamma total
10.72443100000055
coch1
3.3197609999988344
coch2
3.454533000000083
get avg
0.07419099999970058
fftfilter
8.922816000000239
gamma total
11.74392699999953
coch1
3.3234749999992346
coch2
3.4565910000001168
get avg
0.09686999999939871
fftfilter
7.934500999999727
gamma total
10.850243999999293
coch1
3.327848000000813
coch2
3.45576599999913
get avg
0.07323799999903713
fftfilter
7.930705000000671
gamma total
10.865219999999681
coch1
3.3308649999999034
coch2
3.4607230000001437
get avg
0.07517699999880278
fftfilter
8.74261099999967
gamma total
11.481597999998485
coch1
3.3304320000006555
coch2
3.4418230000010226
get avg
0.07485799999994924
fftfilter
9.471553999999742
gamma total
12.377372000000832
coch1
3.2557450000003882
coch2
3.3341549999986455
get avg
0.07644000000072992
fftfilter
9.125810999999885
gamma total
11.915758

In [33]:
sample_data_repo_160 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '160도')
samples_160 = glob.glob(os.path.join(sample_data_repo_160, '**', '*wav'), recursive=True)
samples_160 = sorted(samples_160)   # sort the samples
n_160 = len(samples_160)

for i in range(0, n_160):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_160[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (70 - pred) ** 2

fftfilter
8.760098999999173
gamma total
11.524026000000958
coch1
3.341386000000057
coch2
3.455044000000271
get avg
0.07424399999945308
fftfilter
9.077437999998438
gamma total
11.847625000000335
coch1
3.338315999999395
coch2
3.4226960000014515
get avg
0.07476800000040384
fftfilter
8.921720000000278
gamma total
11.858250999999655
coch1
3.3252560000000813
coch2
3.4510430000009364
get avg
0.10318000000006577
fftfilter
8.338865000001533
gamma total
11.087250999999014
coch1
3.3282060000001366
coch2
3.46603099999993
get avg
0.07415799999944284
fftfilter
9.06493699999919
gamma total
11.800226999999722
coch1
3.350067000001218
coch2
3.4432989999986603
get avg
0.07588900000155263
fftfilter
8.640663999998651
gamma total
11.589578999999503
coch1
3.322469999999157
coch2
3.459494000000632
get avg
0.07406099999934668
fftfilter
8.00065799999902
gamma total
10.90829499999927
coch1
3.3300990000007005
coch2
3.4535759999998845
get avg
0.07828700000027311
fftfilter
8.048909000001004
gamma total
10.992537000

fftfilter
8.85906199999954
gamma total
11.848566000000574
coch1
3.3167610000000423
coch2
3.4584759999997914
get avg
0.10408899999856658
fftfilter
8.668623999999909
gamma total
11.395657000000938
coch1
3.322543999998743
coch2
3.4623670000000857
get avg
0.07407899999998335
fftfilter
9.126368999999613
gamma total
11.986871000000974
coch1
3.334047999998802
coch2
3.469806000000972
get avg
0.10748199999943608
fftfilter
8.388372999999774
gamma total
11.112842000000455
coch1
3.324581999999282
coch2
3.449657000001025
get avg
0.07475800000065647
fftfilter
9.131267999999181
gamma total
11.947771999999532
coch1
3.326795000000857
coch2
3.4452519999995275
get avg
0.07513099999960104
fftfilter
8.77182299999913
gamma total
11.50072099999852
coch1
3.3627160000014555
coch2
3.43169399999897
get avg
0.10340899999937392
fftfilter
9.211663000000044
gamma total
12.032344000001103
coch1
3.3511779999989812
coch2
3.4394809999994322
get avg
0.07893299999886949
fftfilter
8.62499200000093
gamma total
11.3491899999

In [34]:
sample_data_repo_180 = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', '180도')
samples_180 = glob.glob(os.path.join(sample_data_repo_180, '**', '*wav'), recursive=True)
samples_180 = sorted(samples_180)   # sort the samples
n_180 = len(samples_180)

for i in range(0, n_180):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_180[i])
    if(pred == 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)
    else:
        sse = sse + (90 - pred) ** 2

fftfilter
5.411354999998366
gamma total
7.292944999999236
coch1
2.527853999999934
coch2
3.2068230000004405
get avg
0.022988000000623288
fftfilter
5.335721999999805
gamma total
7.229288999998971
coch1
2.22650700000122
coch2
3.0069100000000617
get avg
0.19743999999991502
fftfilter
8.823502000001099
gamma total
11.751936000000569
coch1
3.3111970000009023
coch2
3.444335999998657
get avg
0.134648000001107
fftfilter
9.294167000000016
gamma total
12.050842000000557
coch1
3.2967199999984587
coch2
3.4306670000005397
get avg
0.07262900000023365
fftfilter
11.787802000000738
gamma total
15.633106000001135
coch1
3.496694999999818
coch2
3.674998999998934
get avg
0.20241600000008475
fftfilter
12.358872000000702
gamma total
16.335637000000133
coch1
3.494023000001107
coch2
3.7111129999993864
get avg
0.15639100000043982
fftfilter
9.229522999999972
gamma total
12.232937999999194
coch1
3.3293549999998504
coch2
3.450077000001329
get avg
0.0751139999993029
fftfilter
8.757889000000432
gamma total
11.51706000

In [35]:
sample_data_repo_noise = os.path.join('/home/user3/anaconda3/', 'Data', 'sample_data', 'test', 'noise')
samples_noise = glob.glob(os.path.join(sample_data_repo_noise, '**', '*wav'), recursive=True)
samples_noise = sorted(samples_noise)   # sort the samples
n_noise = len(samples_noise)

for i in range(0, n_noise):
    pred = predict(model_stft_reg, model_stft_clf, model_mrcg_reg, model_mrcg_clf, file_path=samples_noise[i])
    if(pred != 0):
        sse = sse + 180 ** 2
        cnt = cnt + 1
        print(cnt)

fftfilter
8.608939000001556
gamma total
11.361851999999999
coch1
3.322263000000021
coch2
3.4611559999993915
get avg
0.07529000000067754
fftfilter
8.253169999999955
gamma total
10.96516100000008
coch1
3.3246970000000147
coch2
3.4432120000001305
get avg
0.07970200000090699
fftfilter
8.824886999998853
gamma total
11.619171000000279
coch1
3.29414099999849
coch2
3.421064000000115
get avg
0.07421700000122655
fftfilter
8.761153999999806
gamma total
11.5178820000001
coch1
3.329392999999982
coch2
3.4410269999989396
get avg
0.1124260000015056
fftfilter
8.791161999999531
gamma total
11.575200999999652
coch1
3.314027000000351
coch2
3.447526999998445
get avg
0.07555599999977858
fftfilter
8.77263099999982
gamma total
11.730122999999367
coch1
3.345470000000205
coch2
3.461389000000054
get avg
0.10886299999947369
fftfilter
9.173682000000554
gamma total
12.173962000000756
coch1
3.350829999999405
coch2
3.46692400000029
get avg
0.11212999999952444
fftfilter
8.84705600000052
gamma total
11.734972000000198


fftfilter
9.081736000000092
gamma total
12.035068999999567
coch1
3.3245530000003782
coch2
3.4131969999998546
get avg
0.07698200000049837
fftfilter
9.127268000000186
gamma total
11.900158000000374
coch1
3.3237389999994775
coch2
3.4465589999999793
get avg
0.10039099999994505
fftfilter
9.414867999999842
gamma total
12.415941999999632
coch1
3.3121030000002065
coch2
3.4316780000008293
get avg
0.07465700000102515
fftfilter
8.10922600000049
gamma total
11.05479700000069
coch1
3.312942999998995
coch2
3.464428000001135
get avg
0.07463000000097963
fftfilter
8.76979399999982
gamma total
11.696781000000556
coch1
3.3252709999997023
coch2
3.4450010000000475
get avg
0.07593600000109291
fftfilter
8.524214999999458
gamma total
11.28953200000069
coch1
3.3259719999987283
coch2
3.4567889999998442
get avg
0.0751910000017233
fftfilter
8.776214999999866
gamma total
11.504383000001326
coch1
3.330034999999043
coch2
3.4669670000002952
get avg
0.0758270000005723
fftfilter
9.209405999999944
gamma total
12.1780410

In [100]:
# cnt, sse
n = n_0 + n_20 + n_40 + n_60 + n_80 + n_100 + n_120 + n_140 + n_160 + n_180
mse = sse / n
rmse = np.sqrt(mse)
cnt, rmse

(1, 16.59986613065164)

In [23]:
n = n_0 + n_20 + n_40 + n_60 + n_80 + n_100 + n_120 + n_140 + n_160 + n_180
mse = sse / n
rmse = np.sqrt(mse)
cnt, rmse

(2, 30.488092913854246)

In [23]:
n = n_0 + n_20 + n_40 + n_60 + n_80 + n_100 + n_120 + n_140 + n_160 + n_180
mse = sse / n
rmse = np.sqrt(mse)
cnt, rmse

(2, 30.488092913854246)

In [42]:
n = n_0 + n_20 + n_40 + n_60 + n_80 + n_100 + n_120 + n_140 + n_160 + n_180
mse = sse / n
rmse = np.sqrt(mse)
cnt, rmse

(1, 24.918916127158923)

In [59]:
n = n_0 + n_20 + n_40 + n_60 + n_80 + n_100 + n_120 + n_140 + n_160 + n_180
mse = sse / n
rmse = np.sqrt(mse)
cnt, rmse

(1, 53.226082638269304)

In [36]:
n = n_0 + n_20 + n_40 + n_60 + n_80 + n_100 + n_120 + n_140 + n_160 + n_180
mse = sse / n
rmse = np.sqrt(mse)
cnt, rmse

(1, 14.20931954404439)

stft(reg) + mrcg(reg) : (1, 16.59986613065164)                    
stft(clf) + mrcg(clf) : (2, 30.488092913854246)               
stft(reg) + mrcg(clf) : (2, 30.488092913854246)                    
stft(reg) + stft(clf) : (1, 24.918916127158923)                   
stft(reg) + stft(reg) : (1, 53.226082638269304)                  
전체 앙상블 : (1, 14.20931954404439)